# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [ ]:
import torch; import sklearn; from torch import nn
from sklearn.model_selection import train_test_split

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X = iris.data        # 4 features: sépalas e pétalas
y = (iris.target == 1).astype(float)  # 1 se Versicolor, 0 caso contrário

# 2. Preparar dados para pytorch e dividir em treino e teste
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import roc_curve, auc

# Get the predicted probabilities for the positive class
with torch.no_grad():
    outputs = modelo(X_test)
    probabilities = torch.sigmoid(outputs).numpy().flatten()

# Calculate the ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test.numpy(), probabilities)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

## Definição do modelo e treinamento

In [ ]:
# 3. Definir modelo
import torch.nn.functional as F
class RedeNeural(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RedeNeural, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Criar modelo
modelo = RedeNeural(4, 8, 1)  # 4 features → 1 saída (probabilidade de ser Versicolor)

import copy
modelo_clonado = copy.deepcopy(modelo)

learning_rate = 0.1

# Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()  # combinação de sigmoid + BCE
optimizer = torch.optim.SGD(modelo.parameters(), lr=learning_rate)

## Execução do treinamento com optimizer SGD

In [ ]:
# Loop de treino
for epoch in range(1000):
    optimizer.zero_grad()           # Limpa gradientes
    outputs = modelo(X_train)             # Forward
    loss = funcao_perda(outputs, y_train) # Calcula perda
    loss.backward()                 # Calcula derivadas do gradiente
    optimizer.step()                # Aplica regra de alteração dos parâmetros

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

# Treino com regra de gradiente descendente manual

In [ ]:
for epoch in range(1000):
    optimizer.zero_grad()           # Limpa gradientes
    outputs = modelo_clonado(X_train)     # Forward
    loss = funcao_perda(outputs, y_train) # Calcula perda
    loss.backward()                 # Calcula derivadas do gradiente

    with torch.no_grad():
        for param in modelo_clonado.parameters():
            param -= learning_rate * param.grad  # regra de atualização de pesos

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/1000], Loss: {loss.item():.4f}")